# OC4IDS Quality Criteria, Checks and Metrics

Use this notebook to check data against the OC4IDS publication criteria, to perform quality checks and to calculate quality metrics. After running checks, use the status report notebook to report on the results.

## How to use this notebook

1. Run the cells in [Setup](#scrollTo=wh6V7iqi76GR)
2. [Choose a load](#scrollTo=U4apQasSWqjv) to check
3. Run all cells in [Run checks](#scrollTo=OPnM-ojl78gX)

## Setup

In [ ]:
# @title ### Enter database credentials
# @markdown ODS users: Enter the password for the `postgres` user, from the ODS password database.
import getpass

print('Enter your credentials')
user = 'postgres'
password = getpass.getpass('Password:')

In [ ]:
# @title ### Setup notebook environment

!pip install --upgrade ipython-sql > pip.log

connection_string = 'postgresql://' + user + ':' + password + '@oc4ids-database-2.cuujgua4wses.us-east-1.rds.amazonaws.com/postgres'

# https://pypi.org/project/ipython-sql/
%load_ext sql
%sql $connection_string
%config SqlMagic.autopandas = True  # Return Pandas DataFrames instead of regular result sets
%config SqlMagic.displaycon = False  # Don't show connection string after execute
%config SqlMagic.feedback = False  # Don't print number of rows affected by DML
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

# https://colab.research.google.com/notebooks/data_table.ipynb
%load_ext google.colab.data_table
from google.colab.data_table import DataTable
DataTable.max_columns = 50 # Increase max columns so that dataframes with many columns are rendered as data tables
DataTable.include_index = False # Remove the index from data tables for easier copy-pasting to Google Docs

In [ ]:
# @title ### Define functions

import csv
import requests

def get_csv(url):

  response = requests.get(url)
  content = response.content.decode('utf-8').splitlines(keepends=True)

  return csv.DictReader(content, quotechar='"')

def add_exchange_rates(dates):

  """
  Fetch exchange rates (USD base currency) for dates and add them to the exchange_rates table in the database.

  :dates: A one-dimensional Pandas Dataframe containing dates in YYYY-MM-DD format.
  """

  for date in dates['date']:
    r = requests.get(f'https://openexchangerates.org/api/historical/{date}.json?app_id=a937e272181d4ed494e11fcd1fb83c0a&base=USD')

    if r.status_code == requests.codes.ok:
      rates = r.json()
      pairs = json.dumps(rates['rates'])

      query = f"""

      INSERT INTO exchange_rates
      SELECT
          :date AS date,
          key AS currency,
          value::text::numeric AS rate
      FROM
          json_each('{pairs}')
      WHERE ('{date}', key)
      NOT IN ( SELECT DISTINCT
              date,
              currency
          FROM
              exchange_rates);

      """

      %sql {query}

def calculate_indicator_coverage(run_id, collection_ids, source, indicator, fields):

  query = """

    INSERT INTO indicator_coverage (run_id, collection_id, indicator_source, indicator, fields, successes, checks)
    SELECT
        :run_id AS run_id,
        collection_id,
        :source AS indicator_source,
        :indicator AS indicator,
        jsonb_build_array(ARRAY {fields}) AS fields, SUM(
                CASE WHEN ARRAY {fields} <@ paths THEN
                    1
                ELSE
                    0
                END) AS successes, count(*) AS checks FROM project_fields
            WHERE
                collection_id IN :collection_ids GROUP BY collection_id


  """

  query = query.format(fields = fields)

  %sql {query}

## Choose a load

In [ ]:
# @title ### Get a list of loads (collections of collections)

%%sql

SELECT
    load_id,
    array_agg(id) AS collection_ids,
    min(data_version) AS min_data_version,
    max(data_version) AS max_data_version
FROM
    collection
WHERE
    load_id IS NOT NULL
GROUP BY
    load_id
ORDER BY
    max(data_version) DESC;


### Choose `load_id`s to check and compare to

Some checks compare results between loads. Choose the `load_id`s you want to compare.

In [ ]:
load_id = 'dev_1'

In [ ]:
comparison_load_id = 'dev_2'

In [ ]:
# @title ### Get collection ids

result = %sql select array_agg(id) as collection_ids from collection where load_id = :load_id;
collection_ids = tuple(result['collection_ids'][0])

## Run checks

In [ ]:
# @title Set `run_id`

from datetime import datetime
run_id = datetime.now()

In [ ]:
# @title Populate `run_collection` table

%%sql

INSERT INTO run_collection
SELECT
    :run_id AS run_id,
    id AS collection_id
FROM
    collection
WHERE
    load_id = :load_id;



### criteria_registered

In [ ]:
%%sql

WITH failures AS (
    SELECT
        collection_id AS collection_id,
        json_object_agg(project_id,
        LEFT (project_id, 13)) AS output
    FROM
        projects
    WHERE
        collection_id IN :collection_ids
        AND
        LEFT (project_id,
            13)
        NOT IN (
            SELECT
                prefix
            FROM
                registered_prefixes)
        GROUP BY
            collection_id)
    INSERT INTO check_results (run_id, check_id, collection_id, result, output)
    SELECT
        :run_id AS run_id,
        'criteria_registered' AS check_id,
        collection.id AS collection_id,
        CASE WHEN output IS NOT NULL THEN
            FALSE
        ELSE
            TRUE
        END AS result,
        output
    FROM
        failures
    RIGHT JOIN collection ON failures.collection_id = collection.id
WHERE
    collection.id IN :collection_ids;



### criteria_appropriate

In [ ]:
%%sql

WITH additional_fields AS (
    SELECT
        collection_id,
        jsonb_object_agg(key, json_build_object('count', value -> 'count', 'examples', value -> 'examples')) AS output
    FROM
        collection_check
        CROSS JOIN jsonb_each(cove_output -> 'additional_fields')
    WHERE
        collection_id IN :collection_ids
        AND value -> 'additional_field_descendance' IS NULL
        AND jsonb_array_length(value -> 'examples') > 0
    GROUP BY
        collection_id)
INSERT INTO check_results (run_id, check_id, collection_id, result, output)
SELECT
    :run_id AS run_id,
    'criteria_appropriate' AS check_id,
    collection.id,
    NULL AS result,
    output
FROM
    additional_fields
    RIGHT JOIN collection ON additional_fields.collection_id = collection.id
WHERE
    collection.id IN :collection_ids;



### criteria_active

In [ ]:
%%sql

INSERT INTO check_results (run_id, check_id, collection_id, result, output)
SELECT
    :run_id AS run_id,
    'criteria_active' AS check_id,
    collection_id,
    CASE WHEN max(data ->> 'updated')::date >= CURRENT_DATE - interval '12 month' THEN
        TRUE
    ELSE
        FALSE
    END AS result,
    NULL AS output
FROM
    projects
WHERE
    collection_id IN :collection_ids
GROUP BY
    collection_id;



### criteria_valid

In [ ]:
%%sql

WITH failures AS (
    SELECT
        collection_id,
        count(*) = 0 AS result
    FROM
        collection_check
        CROSS JOIN jsonb_array_elements(cove_output -> 'validation_errors') AS validation_errors
    WHERE
        collection_id IN :collection_ids
    GROUP BY
        collection_id)
INSERT INTO check_results (run_id, check_id, collection_id, result, output)
SELECT
    :run_id AS run_id,
    'criteria_valid' AS check_id,
    collection.id,
    CASE WHEN result IS NOT NULL THEN
        FALSE
    ELSE
        TRUE
    END AS result,
    NULL AS output
FROM
    failures
    RIGHT JOIN collection ON failures.collection_id = collection.id
WHERE
    collection.id IN :collection_ids;



### criteria_conformant

In [ ]:
%%sql
WITH failures AS (
    SELECT
        collection_id,
        jsonb_build_object('count', count(DISTINCT key)) AS output
    FROM
        collection_check
        CROSS JOIN jsonb_each(cove_output -> 'structure_warnings')
    WHERE
        collection_id IN :collection_ids
    GROUP BY
        collection_id)
INSERT INTO check_results (run_id, check_id, collection_id, result, output)
SELECT
    :run_id AS run_id,
    'criteria_conformant' AS check_id,
    collection.id AS collection_id,
    CASE WHEN output IS NOT NULL THEN
        FALSE
    ELSE
        TRUE
    END AS result,
    NULL AS output
FROM
    failures
    RIGHT JOIN collection ON failures.collection_id = collection.id
WHERE
    collection.id IN :collection_ids;



### metrics_new_projects

In [ ]:
%%sql

WITH new_projects AS (
    SELECT
        collection_id,
        jsonb_build_object('count', count(project_id)) AS output
    FROM
        projects
    WHERE
        collection_id IN :collection_ids
        AND project_id NOT IN (
            SELECT
                project_id
            FROM
                projects
            WHERE
                collection_id IN (
                    SELECT
                        id
                    FROM
                        collection
                    WHERE
                        load_id = :comparison_load_id))
            GROUP BY
                collection_id)
    INSERT INTO check_results (run_id, check_id, collection_id, result, output)
SELECT
    :run_id AS run_id,
    'metrics_new_projects' AS check_id,
    collection.id AS collection_id,
    NULL AS result,
    coalesce(output, jsonb_build_object('count', NULL)) AS output
FROM
    new_projects
    RIGHT JOIN collection ON new_projects.collection_id = collection.id
WHERE
    collection.id IN :collection_ids;



### metrics_last_updated

In [ ]:
%%sql

INSERT INTO check_results (run_id, check_id, collection_id, result, output)
SELECT
    :run_id AS run_id,
    'metrics_last_updated' AS check_id,
    collection_id,
    NULL AS result,
    jsonb_build_object('date', max(data ->> 'updated')::date) AS output
FROM
    projects
WHERE
    collection_id IN :collection_ids
GROUP BY
    collection_id;



### metrics_earliest_start_date

In [ ]:
%%sql

INSERT INTO check_results (run_id, check_id, collection_id, result, output)
SELECT
    :run_id AS run_id,
    'metrics_earliest_start_date' AS check_id,
    collection_id,
    NULL AS result,
    jsonb_build_object('date', min(data -> 'period' ->> 'startDate')::date) AS output
FROM
    projects
WHERE
    collection_id IN :collection_ids
GROUP BY
    collection_id;



### metrics_latest_end_date

In [ ]:
%%sql

INSERT INTO check_results (run_id, check_id, collection_id, result, output)
SELECT
    :run_id AS run_id,
    'metrics_latest_end_date' AS check_id,
    collection_id,
    NULL AS result,
    jsonb_build_object('date', max(data -> 'period' ->> 'endDate')::date) AS output
FROM
    projects
WHERE
    collection_id IN :collection_ids
GROUP BY
    collection_id;



### metrics_additional_field_count

In [ ]:
%%sql

WITH counts AS (
    SELECT DISTINCT
        collection_id,
        jsonb_build_object('count', count(DISTINCT key)) AS output
    FROM
        collection_check
        CROSS JOIN jsonb_each(cove_output -> 'additional_fields')
    WHERE
        collection_id IN :collection_ids
    GROUP BY
        collection_id)
INSERT INTO check_results (run_id, check_id, collection_id, result, output)
SELECT
    :run_id AS run_id,
    'metrics_additional_field_count' AS check_id,
    collection.id AS collection_id,
    NULL AS result,
    coalesce(output, jsonb_build_object('count', NULL))
FROM
    counts
    RIGHT JOIN collection ON counts.collection_id = collection.id
WHERE
    collection.id IN :collection_ids;



### metrics_project_count

In [ ]:
%%sql

WITH counts AS (
    SELECT
        collection_id,
        jsonb_build_object('count', count(*)) AS output
    FROM
        projects
    WHERE
        collection_id IN :collection_ids
    GROUP BY
        collection_id)
INSERT INTO check_results (run_id, check_id, collection_id, result, output)
SELECT
    :run_id AS run_id,
    'metrics_project_count' AS check_id,
    collection.id AS collection_id,
    NULL AS result,
    coalesce(output, jsonb_build_object('count', NULL)) AS output
FROM
    counts
    RIGHT JOIN collection ON counts.collection_id = collection.id
WHERE
    collection.id IN :collection_ids;



### metrics_validation_error_count

In [ ]:
%%sql

WITH counts AS (
    SELECT
        collection_id,
        jsonb_build_object('count', count(DISTINCT replace(trim('"' FROM (validation_errors -> 0)::text), '\', '')::jsonb -> 'message')) AS output FROM collection_check
                CROSS JOIN jsonb_array_elements(cove_output -> 'validation_errors') AS validation_errors
                WHERE
                    collection_id IN :collection_ids GROUP BY collection_id)
                INSERT INTO check_results (run_id, check_id, collection_id, result, output)
                SELECT
                    :run_id AS run_id, 'metrics_validation_error_count' AS check_id, collection.id AS collection_id, NULL AS result, coalesce(output, jsonb_build_object('count', NULL))
                    FROM counts
                RIGHT JOIN collection ON counts.collection_id = collection.id
                WHERE
                    collection.id IN :collection_ids;



### metrics_structure_warning_count

In [ ]:
%%sql

WITH counts AS (
    SELECT
        collection_id,
        jsonb_build_object('count', count(DISTINCT key)) AS output
    FROM
        collection_check
        CROSS JOIN jsonb_each(cove_output -> 'structure_warnings')
    WHERE
        collection_id IN :collection_ids
    GROUP BY
        collection_id)
INSERT INTO check_results (run_id, check_id, collection_id, result, output)
SELECT
    :run_id AS run_id,
    'metrics_structure_warning_count' AS check_id,
    collection.id AS collection_id,
    NULL AS result,
    coalesce(output, jsonb_build_object('count', NULL))
FROM
    counts
    RIGHT JOIN collection ON counts.collection_id = collection.id
WHERE
    collection.id IN :collection_ids;



### semantics_sector_codelist

In [ ]:
%%sql

INSERT INTO check_results (run_id, check_id, collection_id, result, output)
SELECT
    :run_id AS run_id,
    'semantics_sector_codelist' AS check_id,
    collection_id AS collection_id,
    CASE WHEN jsonb_array_length(cove_output -> 'additional_open_codelist_values' -> 'projects/sector' -> 'values') > 0 THEN
        FALSE
    ELSE
        TRUE
    END AS result,
    jsonb_build_object('all_projects', cove_output -> 'additional_open_codelist_values' -> 'projects/sector' -> 'values') AS output
FROM
    collection_check
WHERE
    collection_id IN :collection_ids;



### semantics_public_authority_names

In [ ]:
%%sql

WITH names AS (
    SELECT
        collection_id,
        project_id,
        data -> 'publicAuthority' -> 'name' AS name,
        row_number() OVER (PARTITION BY collection_id ORDER BY random()) AS rank
    FROM
        projects
    WHERE
        collection_id IN :collection_ids
        AND data -> 'publicAuthority' -> 'name' IS NOT NULL
),
output AS (
    SELECT
        collection_id,
        json_object_agg(project_id, name) AS output
    FROM
        names
    WHERE
        rank <= 10
    GROUP BY
        collection_id)
INSERT INTO check_results (run_id, check_id, collection_id, result, output)
SELECT
    :run_id AS run_id,
    'semantics_public_authority_names' AS check_id,
    collection.id,
    NULL AS result,
    output
FROM
    output
    RIGHT JOIN collection ON output.collection_id = collection.id
WHERE
    collection.id IN :collection_ids;



### semantics_supplier_names

In [ ]:
%%sql

WITH names AS (
    SELECT
        collection_id,
        project_id,
        suppliers -> 'name' AS name,
        row_number() OVER (PARTITION BY collection_id ORDER BY random()) AS rank
    FROM
        projects
        CROSS JOIN jsonb_array_elements(data -> 'contractingProcesses') AS contractingProcesses
        CROSS JOIN jsonb_array_elements(contractingProcesses -> 'summary' -> 'suppliers') AS suppliers
    WHERE
        collection_id IN :collection_ids
        AND data -> 'contractingProcesses' IS NOT NULL
        AND contractingProcesses -> 'summary' -> 'suppliers' IS NOT NULL
        AND suppliers -> 'name' IS NOT NULL
),
output AS (
    SELECT
        collection_id,
        json_object_agg(project_id, name) AS output
    FROM
        names
    WHERE
        rank <= 10
    GROUP BY
        collection_id)
INSERT INTO check_results (run_id, check_id, collection_id, result, output)
SELECT
    :run_id AS run_id,
    'semantics_supplier_names' AS check_id,
    collection.id,
    NULL AS result,
    output
FROM
    output
    RIGHT JOIN collection ON output.collection_id = collection.id
WHERE
    collection.id IN :collection_ids;



### semantics_budgets

Get previously unseen dates:

In [ ]:
%%sql dates <<

SELECT DISTINCT
    coalesce(data -> 'budget' ->> 'approvalDate', data -> 'period' ->> 'startDate')::date AS date
FROM
    projects
WHERE
    collection_id IN :collection_ids
    AND data -> 'period' -> 'startDate' IS NOT NULL
    AND data -> 'budget' -> 'amount' -> 'currency' IS NOT NULL
    AND data -> 'budget' -> 'amount' -> 'amount' IS NOT NULL
    AND data -> 'budget' -> 'amount' ->> 'currency' != 'USD'
    AND coalesce(data -> 'budget' ->> 'approvalDate', data -> 'period' ->> 'startDate')::date NOT IN ( SELECT DISTINCT
            date
        FROM
            exchange_rates)
ORDER BY
    date ASC;



Add exchange rates to database:

In [ ]:
add_exchange_rates(dates)

Run check:

In [ ]:
%%sql

WITH numeric_budgets AS (
    SELECT
        collection_id,
        project_id,
        coalesce(data -> 'budget' ->> 'approvalDate', data -> 'period' ->> 'startDate')::date AS date,
        (data -> 'budget' -> 'amount' -> 'amount')::numeric AS amount,
        data -> 'budget' -> 'amount' ->> 'currency' AS currency
    FROM
        projects
    WHERE
        collection_id IN :collection_ids
        AND jsonb_typeof(DATA -> 'budget' -> 'amount' -> 'amount') = 'number'
),
usd_budgets AS (
    SELECT
        collection_id,
        project_id,
        amount / exchange_rates.rate AS amount_usd
    FROM
        numeric_budgets
        JOIN exchange_rates ON numeric_budgets.date = exchange_rates.date
            AND numeric_budgets.currency = exchange_rates.currency
),
failures AS (
    SELECT
        collection_id,
        json_object_agg(project_id, amount_usd) AS output
    FROM
        usd_budgets
    WHERE
        amount_usd <= 0
        OR amount_usd >= 5000000000
    GROUP BY
        collection_id)
    -- insert into
    --   check_results (run_id, check_id, collection_id, result, output)
    SELECT
        :run_id AS run_id,
        'semantics_budgets' AS check_id,
        collection.id AS collection_id,
        CASE WHEN output IS NOT NULL THEN
            FALSE
        ELSE
            TRUE
        END AS result,
        output
    FROM
        failures
    RIGHT JOIN collection ON failures.collection_id = collection.id
WHERE
    collection.id IN :collection_ids;



### semantics_contract_values

Get previously unseen dates:

In [ ]:
%%sql dates <<

SELECT DISTINCT
    coalesce(contracting_processes -> 'summary' -> 'contractPeriod' ->> 'startDate', contracting_processes -> 'summary' -> 'tender' ->> 'datePublished')::date AS date
FROM
    projects
    CROSS JOIN jsonb_array_elements(data -> 'contractingProcesses') AS contracting_processes
WHERE
    collection_id IN :collection_ids
    AND contracting_processes -> 'summary' -> 'contractValue' -> 'currency' IS NOT NULL
    AND contracting_processes -> 'summary' -> 'contractValue' -> 'amount' IS NOT NULL
    AND contracting_processes -> 'summary' -> 'contractValue' ->> 'currency' != 'USD'
    AND coalesce(contracting_processes -> 'summary' -> 'contractPeriod' ->> 'startDate', contracting_processes -> 'summary' -> 'tender' ->> 'datePublished')::date NOT IN ( SELECT DISTINCT
            date
        FROM
            exchange_rates)
ORDER BY
    date ASC;



Add exchange rates to database:

In [ ]:
add_exchange_rates(dates)

Run check:

In [ ]:
%%sql

WITH numeric_values AS (
    SELECT
        collection_id,
        project_id,
        coalesce(contracting_processes -> 'summary' -> 'contractPeriod' ->> 'startDate', contracting_processes -> 'summary' -> 'tender' ->> 'datePublished')::date AS date,
        (contracting_processes -> 'summary' -> 'contractValue' -> 'amount')::numeric AS amount,
        contracting_processes -> 'summary' -> 'contractValue' ->> 'currency' AS currency
    FROM
        projects
        CROSS JOIN jsonb_array_elements(data -> 'contractingProcesses') AS contracting_processes
    WHERE
        collection_id IN :collection_ids
        AND jsonb_typeof(contracting_processes -> 'summary' -> 'contractValue' -> 'amount') = 'number'
),
usd_values AS (
    SELECT
        collection_id,
        project_id,
        amount / exchange_rates.rate AS amount_usd
    FROM
        numeric_values
        JOIN exchange_rates ON numeric_values.date = exchange_rates.date
            AND numeric_values.currency = exchange_rates.currency
),
failures AS (
    SELECT
        collection_id,
        json_object_agg(project_id, amount_usd) AS output
FROM
    usd_values
WHERE
    amount_usd <= 0
    OR amount_usd >= 5000000000
GROUP BY
    collection_id)
    INSERT INTO check_results (run_id, check_id, collection_id, result, output)
    SELECT
        :run_id AS run_id,
        'semantics_contract_values' AS check_id,
        collection.id AS collection_id,
        CASE WHEN output IS NOT NULL THEN
            FALSE
        ELSE
            TRUE
        END AS result,
        output
    FROM
        failures
    RIGHT JOIN collection ON failures.collection_id = collection.id
WHERE
    collection.id IN :collection_ids;



### semantics_funder_names

In [ ]:
%%sql

WITH names AS (
    SELECT
        collection_id,
        project_id,
        parties -> 'name' AS name,
        row_number() OVER (PARTITION BY collection_id ORDER BY random()) AS rank
    FROM
        projects
        CROSS JOIN jsonb_array_elements(data -> 'parties') AS parties
    WHERE
        collection_id IN :collection_ids
        AND data -> 'parties' IS NOT NULL
        AND parties -> 'name' IS NOT NULL
        AND parties -> 'roles' ? 'funder'
),
output AS (
    SELECT
        collection_id,
        json_object_agg(project_id, name) AS output
    FROM
        names
    WHERE
        rank <= 10
    GROUP BY
        collection_id)
INSERT INTO check_results (run_id, check_id, collection_id, result, output)
SELECT
    :run_id AS run_id,
    'semantics_funder_names' AS check_id,
    collection.id,
    NULL AS result,
    output
FROM
    output
    RIGHT JOIN collection ON output.collection_id = collection.id
WHERE
    collection.id IN :collection_ids;



### semantics_dates

In [ ]:
%%sql

WITH dates AS (
    SELECT
        collection_id,
        project_id,
        'updated' AS path,
        data ->> 'updated' AS value
    FROM
        projects
    WHERE
        collection_id IN :collection_ids
        AND data -> 'updated' IS NOT NULL
        AND data ->> 'updated' != ''
    UNION
    SELECT
        collection_id,
        project_id,
        'period/startDate' AS path,
        data -> 'period' ->> 'startDate' AS value
    FROM
        projects
    WHERE
        collection_id IN :collection_ids
        AND data -> 'period' -> 'startDate' IS NOT NULL
        AND data -> 'period' ->> 'startDate' != ''
    UNION
    SELECT
        collection_id,
        project_id,
        'period/endDate' AS path,
        data -> 'period' ->> 'endDate' AS value
    FROM
        projects
    WHERE
        collection_id IN :collection_ids
        AND data -> 'period' -> 'endDate' IS NOT NULL
        AND data -> 'period' ->> 'endDate' != ''
    UNION
    SELECT
        collection_id,
        project_id,
        'completion/endDate' AS path,
        data -> 'completion' ->> 'endDate' AS value
    FROM
        projects
    WHERE
        collection_id IN :collection_ids
        AND data -> 'completion' IS NOT NULL
        AND data -> 'completion' ->> 'endDate' != ''
),
failures AS (
    SELECT
        collection_id,
        json_object_agg(project_id, value) AS output
FROM
    dates
    WHERE
        value::date <= '1970-01-01'::date
        OR value::date >= '2050-01-01'::date
    GROUP BY
        collection_id)
INSERT INTO check_results (run_id, check_id, collection_id, result, output)
SELECT
    :run_id AS run_id,
    'semantics_dates' AS check_id,
    collection.id,
    CASE WHEN output IS NOT NULL THEN
        FALSE
    ELSE
        TRUE
    END AS result,
    output
FROM
    failures
    RIGHT JOIN collection ON failures.collection_id = collection.id
WHERE
    collection.id IN :collection_ids;



### semantics_role_coherence

In [ ]:
%%sql

WITH missing_roles AS (
    -- publicAuthority
    SELECT
        collection_id,
        project_id,
        'publicAuthority' AS role
    FROM
        projects
        JOIN jsonb_array_elements(data -> 'parties') AS parties ON parties -> 'id' = data -> 'publicAuthority' -> 'id'
    WHERE
        collection_id IN :collection_ids
        AND NOT parties -> 'roles' ? 'publicAuthority'
    UNION ALL
    -- budget/sourceParty
    SELECT
        collection_id,
        project_id,
        'sourceParty' AS role
    FROM
        projects
        JOIN jsonb_array_elements(data -> 'parties') AS parties ON parties -> 'id' = data -> 'budget' -> 'sourceParty' -> 'id'
    WHERE
        collection_id IN :collection_ids
        AND NOT parties -> 'roles' ? 'sourceParty'
    UNION ALL
    -- contractingProcesses/summary/tender/tenderers
    SELECT
        collection_id,
        project_id,
        'tenderer' AS role
    FROM
        projects
        JOIN jsonb_array_elements(data -> 'parties') AS parties
        CROSS JOIN jsonb_array_elements(data -> 'contractingProcesses') AS contracting_processes
        CROSS JOIN jsonb_array_elements(contracting_processes -> 'summary' -> 'tender' -> 'tenderers') AS tenderers ON parties -> 'id' = tenderers -> 'id'
    WHERE
        collection_id IN :collection_ids
        AND NOT parties -> 'roles' ? 'tenderer'
    UNION ALL
    -- contractingProcesses/summary/tender/procuringEntity
    SELECT
        collection_id,
        project_id,
        'procuringEntity' AS role
    FROM
        projects
        JOIN jsonb_array_elements(data -> 'parties') AS parties
        CROSS JOIN jsonb_array_elements(data -> 'contractingProcesses') AS contracting_processes ON parties -> 'id' = contracting_processes -> 'summary' -> 'tender' -> 'procuringEntity' -> 'id'
    WHERE
        collection_id IN :collection_ids
        AND NOT parties -> 'roles' ? 'procuringEntity'
    UNION ALL
    -- contractingProcesses/summary/tender/administrativeEntity
    SELECT
        collection_id,
        project_id,
        'administrativeEntity' AS role
    FROM
        projects
        JOIN jsonb_array_elements(data -> 'parties') AS parties
        CROSS JOIN jsonb_array_elements(data -> 'contractingProcesses') AS contracting_processes ON parties -> 'id' = contracting_processes -> 'summary' -> 'tender' -> 'administrativeEntity' -> 'id'
    WHERE
        collection_id IN :collection_ids
        AND NOT parties -> 'roles' ? 'administrativeEntity'
    UNION ALL
    -- contractingProcesses/summary/suppliers
    SELECT
        collection_id,
        project_id,
        'supplier' AS role
    FROM
        projects
        JOIN jsonb_array_elements(data -> 'parties') AS parties
        CROSS JOIN jsonb_array_elements(data -> 'contractingProcesses') AS contracting_processes
        CROSS JOIN jsonb_array_elements(contracting_processes -> 'summary' -> 'suppliers') AS suppliers ON parties -> 'id' = suppliers -> 'id'
    WHERE
        collection_id IN :collection_ids
        AND NOT parties -> 'roles' ? 'supplier'
),
failures AS (
    SELECT
        collection_id,
        json_object_agg(project_id, ROLE) AS output
    FROM
        missing_roles
    GROUP BY
        collection_id)
    INSERT INTO check_results (run_id, check_id, collection_id, result, output)
    SELECT
        :run_id AS run_id,
        'semantics_role_coherence' AS check_id,
        collection.id AS collection_id,
        CASE WHEN output IS NOT NULL THEN
            FALSE
        ELSE
            TRUE
        END AS result,
        output
    FROM
        failures
    RIGHT JOIN collection ON failures.collection_id = collection.id
WHERE
    collection.id IN :collection_ids;



### semantics_coordinates

In [ ]:
%%sql

WITH invalid_coordinates AS (
    SELECT
        collection_id,
        project_id,
        locations -> 'geometry' -> 'coordinates' AS coordinates
    FROM
        projects
        CROSS JOIN jsonb_array_elements(data -> 'locations') AS locations
    WHERE
        collection_id IN :collection_ids
        AND locations -> 'geometry' ->> 'type' = 'Point'
        AND ((locations -> 'geometry' -> 'coordinates' ->> 0)::numeric NOT BETWEEN -90 AND 90
            OR (locations -> 'geometry' -> 'coordinates' ->> 1)::numeric NOT BETWEEN -180 AND 180)
    UNION ALL
    SELECT
        collection_id,
        project_id,
        locations -> 'geometry' -> 'coordinates' AS coordinates
    FROM
        projects
        CROSS JOIN jsonb_array_elements(data -> 'locations') AS locations
        CROSS JOIN jsonb_array_elements(locations -> 'geometry' -> 'coordinates') AS coordinates
    WHERE
        collection_id IN :collection_ids
        AND locations -> 'geometry' -> 'type' ?| ARRAY['Multipoint',
        'LineString']
        AND ((coordinates ->> 0)::numeric NOT BETWEEN -90 AND 90
            OR (coordinates ->> 1)::numeric NOT BETWEEN -180 AND 180)
    UNION ALL
    SELECT
        collection_id,
        project_id,
        locations -> 'geometry' -> 'coordinates' AS coordinates
    FROM
        projects
        CROSS JOIN jsonb_array_elements(data -> 'locations') AS locations
        CROSS JOIN jsonb_array_elements(locations -> 'geometry' -> 'coordinates') AS linear_rings
        CROSS JOIN jsonb_array_elements(linear_rings) AS coordinates
    WHERE
        collection_id IN :collection_ids
        AND locations -> 'geometry' -> 'type' ?| ARRAY['Polygon',
        'MultiLineString']
        AND ((coordinates ->> 0)::numeric NOT BETWEEN -90 AND 90
            OR (coordinates ->> 1)::numeric NOT BETWEEN -180 AND 180)
    UNION ALL
    SELECT
        collection_id,
        project_id,
        locations -> 'geometry' -> 'coordinates' AS coordinates
    FROM
        projects
        CROSS JOIN jsonb_array_elements(data -> 'locations') AS locations
        CROSS JOIN jsonb_array_elements(locations -> 'geometry' -> 'coordinates') AS polygons
        CROSS JOIN jsonb_array_elements(polygons) AS linear_rings
        CROSS JOIN jsonb_array_elements(linear_rings) AS coordinates
    WHERE
        collection_id IN :collection_ids
        AND locations -> 'geometry' ->> 'type' = 'Multipolygon'
        AND ((coordinates ->> 0)::numeric BETWEEN -90 AND 90
            OR (coordinates ->> 1)::numeric NOT BETWEEN -180 AND 180)
),
failures AS (
    SELECT
        collection_id,
        json_object_agg(project_id, coordinates) AS output
    FROM
        invalid_coordinates
    GROUP BY
        collection_id)
    INSERT INTO check_results (run_id, check_id, collection_id, result, output)
    SELECT
        :run_id AS run_id,
        'semantics_coordinates' AS check_id,
        collection.id AS collection_id,
        CASE WHEN output IS NOT NULL THEN
            FALSE
        ELSE
            TRUE
        END AS result,
        output
    FROM
        failures
    RIGHT JOIN collection ON failures.collection_id = collection.id
WHERE
    collection.id IN :collection_ids;



### coverage_cost_ids

In [ ]:
base_url = 'https://standard.open-contracting.org/staging/infrastructure/0.9-dev/en/_static/project-level/'

csv_files = [
  'process-level-implementation.csv',
  'process-level-procurement.csv',
  'project-level-completion.csv',
  'project-level-identification.csv',
  'project-level-preparation.csv',
  'reactive-process-level-contract.csv',
  'reactive-process-level-implementation.csv',
  'reactive-process-level-procurement.csv',
  'reactive-project-level-completion.csv',
  'reactive-project-level-identification-preparation.csv'
        ]

for url in [f'{base_url}{filename}' for filename in csv_files]:
  mapping_reader = get_csv(url)

  for row in mapping_reader:

    if len(row['OC4IDS Fields']) > 0:

      calculate_indicator_coverage(
        run_id = run_id,
        collection_ids = collection_ids,
        source = 'cost_ids',
        indicator = row['CoST IDS element'],
        fields = row['OC4IDS Fields'].split(',')
      )

### coverage_sustainability_modules

In [ ]:
url = 'https://standard.open-contracting.org/staging/infrastructure/0.9-dev/en/_static/sustainability.yaml'

response = requests.get(url, allow_redirects=True)
content = response.content.decode("utf-8")
indicators = yaml.safe_load(content)

for indicator in indicators:

  calculate_indicator_coverage(
    run_id = run_id,
    collection_ids = collection_ids,
    source = 'sustainability_modules',
    indicator = f"{indicator['id']}: {indicator['title']}",
    fields = indicator['fields']
  )
